# 코스 선택 (랜덤)

# lang graph skeleton
1. 뼈대 정의 (State + Graph)

QuizState 타입 정의

노드 이름(NoviceQuiz, IntermediateQuiz, ExpertQuiz)과 기본 동작 (graph.add_node)

엣지 연결 (graph.add_edge)

엔트리 포인트 (graph.set_entry_point)

In [3]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Dict

# === 상태 정의 ===
class QuizState(TypedDict):
    stage: str
    result: Dict

# === Novice Subnodes ===
def background_knowledge(state: QuizState) -> QuizState:
    state["result"]["background"] = "배경지식 생성"
    return state

def novice_quiz(state: QuizState) -> QuizState:
    state["result"]["novice_quiz"] = "초급 퀴즈 생성"
    return state

# === Intermediate Subnodes ===
def intermediate_quiz(state: QuizState) -> QuizState:
    state["result"]["intermediate_quiz"] = "중급 퀴즈 생성"
    return state

def reflection(state: QuizState) -> QuizState:
    state["result"]["reflection"] = "Reflection 실행"
    return state

# === Expert Subnodes ===
def expert_quiz(state: QuizState) -> QuizState:
    state["result"]["expert_quiz"] = "고급 퀴즈 생성"
    return state

# === 그래프 생성 ===
graph = StateGraph(QuizState)

# Novice Stage
graph.add_node("Background", background_knowledge)
graph.add_node("NoviceQuiz", novice_quiz)

# Intermediate Stage
graph.add_node("IntermediateQuiz", intermediate_quiz)
graph.add_node("IntermediateReflection", reflection)

# Expert Stage
graph.add_node("ExpertQuiz", expert_quiz)
graph.add_node("ExpertReflection", reflection)

# === Novice 흐름 (병렬 실행) ===
graph.add_edge("Background", END)     # 배경지식은 독립 실행
graph.add_edge("NoviceQuiz", END)     # 퀴즈도 독립 실행

# === Intermediate 흐름 (직렬) ===
graph.add_edge("IntermediateQuiz", "IntermediateReflection")
graph.add_edge("IntermediateReflection", END)

# === Expert 흐름 (직렬) ===
graph.add_edge("ExpertQuiz", "ExpertReflection")
graph.add_edge("ExpertReflection", END)

# === Entry Point ===
graph.set_entry_point("Background")  # 또는 "NoviceQuiz"로 시작 가능

app = graph.compile()

# === 실행 예시 ===
initial_state: QuizState = {"stage": "novice", "result": {}}
final_state = app.invoke(initial_state)
print(final_state["result"])


{'background': '배경지식 생성'}


# n 단계 배경지식 노드

In [ ]:
import os
import json
import requests
from openai import OpenAI
from pathlib import Path
from dotenv import load_dotenv

# === 환경 변수 로드 ===
env_path = Path("..") / ".env"
load_dotenv(dotenv_path=env_path)

GOOGLE_API_KEY = os.getenv("GOOGLE_CSE_API_KEY")
CX_NEWS = os.getenv("GOOGLE_CSE_CX_NEWS")
CX_GOV = os.getenv("GOOGLE_CSE_CX_GOV")

client = OpenAI()

# === 기사 불러오기 ===
json_file = Path("..") / "data" / "course" / "politics_clustered.json"
with open(json_file, "r", encoding="utf-8") as f:
    courses = json.load(f)

selected_course = courses[2]
articles_sorted = sorted(selected_course["articles"], key=lambda x: x["published_at"], reverse=True)
latest_article = articles_sorted[0]

# === LLM 프롬프트 ===
prompt = f"""
당신은 언론 분석가입니다. 아래 기사 요약문을 읽고, 
독자가 이해를 돕기 위해 필요한 배경지식을 항목별로 정리하세요.

항목:
- 이슈명 (기사 주제·핵심 키워드)
- 원인 (사건 발생 배경·원인 요인)
- 상황(타임라인) (시점별 전개·주요 행위자)
- 결과 (단기적 결과·즉각적 변화)
- 영향 (장기적 사회·경제·정치적 함의)

각 항목의 summary는 한 문장으로 간결히 요약하세요.
각 query는 실제 Google 검색용으로 사용될 키워드이므로, 핵심 명사 위주로 작성하세요.

출력은 반드시 JSON만 출력하세요.  
설명, 인사말, 따옴표 밖 텍스트 등은 절대 포함하지 마세요.

출력 형식:
{{
  "background": [
    {{"type": "이슈명", "summary": "...", "query": "..." }},
    {{"type": "원인", "summary": "...", "query": "..." }},
    {{"type": "상황(타임라인)", "summary": "...", "query": "..." }},
    {{"type": "결과", "summary": "...", "query": "..." }},
    {{"type": "영향", "summary": "...", "query": "..." }}
  ]
}}

기사 요약문:
{latest_article.get("summary")}
"""

# === LLM 호출 ===
resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}]
)


try:
    background_info = json.loads(raw_output)
except Exception as e:
    print("JSON 파싱 실패:", e)
    # 정규식으로 JSON만 추출 (혹시 텍스트 섞였을 경우)
    import re
    match = re.search(r"\{.*\}", raw_output, re.DOTALL)
    if match:
        try:
            background_info = json.loads(match.group(0))
        except Exception:
            background_info = {"background": []}
    else:
        background_info = {"background": []}

# === Google CSE 검색 함수 ===
def google_search(query, cx):
    url = "https://www.googleapis.com/customsearch/v1"
    params = {"key": GOOGLE_API_KEY, "cx": cx, "q": query, "num": 1}
    try:
        r = requests.get(url, params=params)
        r.raise_for_status()
        data = r.json()
        if "items" in data and len(data["items"]) > 0:
            item = data["items"][0]
            return {"title": item["title"], "link": item["link"]}
        else:
            return None
    except Exception as e:
        print(f"검색 실패 ({query}):", e)
        return None

# === 배경지식별 URL 검색 ===
for b in background_info.get("background", []):
    query = b.get("query")
    if not query:
        b["url"] = None
        continue

    # 뉴스 → 정부 순서로 검색 시도
    result = google_search(query, CX_NEWS) or google_search(query, CX_GOV)
    b["url"] = result["link"] if result else None

# === 최종 JSON 출력 ===
final_output = {
    "articles": [latest_article],
    "background": background_info.get("background", [])
}

print("\n=== 최종 출력 ===\n")
print(json.dumps(final_output, ensure_ascii=False, indent=2))


=== 최종 출력 ===

{
  "articles": [
    {
      "id": "1615114173493154068",
      "sections": [
        "politics"
      ],
      "title": "장동혁 \"특검 압수수색, 최소한의 증거만 임의제출 되도록 협의\"",
      "publisher": "뉴스1",
      "author": "홍유진",
      "summary": "\"협의 사항에 대해 따로 보고 받은 바 없어\" \"공정노사법, 당론으로 추진할지는 원내대표와 협의할 것\" (서울=뉴스1) 박소은 홍유진 기자 = 장동혁 국민의힘 대표는 4일 12·3 비상계엄 관련 내란·외환 사건을 수사하는 조은석 특별검사팀이 자당의 원내대표실과 원내행정국 압수수색을 시도하는 것에 대해 \"최소한의 범위에서, 최소한의 증거들만 임의제출 될 수 있도록 협의해 나가겠다\"고 강조했다.\n\n법원에서도 특별재판부, 인민재판부에 대해서는 심각한 우려를 표하고 있다\"며 \"대한민국이 민주화 이후에, 그리고 지금의 헌법을 가진 이후에 헌법 규정에도 없는 특별재판부를 만든다고 하는 상상 자체가 믿기 어렵다\"고 했다.\n\n그러면서 \"결국 특검이 성과 내지 못하고, 그간의 내란 정당 몰이가 성과 없이 끝나게 될 것으로  ...",
      "highlight": null,
      "score": null,
      "image_url": "https://ddi-cdn.deepsearch.com/news/politics/2025/09/04/1615114173493154068/000-9ecc9ec464b060c96c9ad3a10cebd95016014bbf.jpg",
      "thumbnail_url": "https://ddi-cdn.deepsearch.com/news_thumbnail/politics/2025/09/04/1615114173493154068/000-9ecc9ec464b060c96c9ad3a10c

# n 단계 node 퀴즈 노드 실행

In [9]:
import json
from openai import OpenAI
from pathlib import Path

client = OpenAI()

# === 기사 불러오기 ===
json_file = Path("..") / "data" / "course" / "politics_clustered.json"
with open(json_file, "r", encoding="utf-8") as f:
    courses = json.load(f)

selected_course = courses[2]
articles_sorted = sorted(selected_course["articles"], key=lambda x: x["published_at"], reverse=True)
latest_article = articles_sorted[0]

# === 프롬프트 ===
prompt = f"""
아래 기사 요약문을 기반으로 퀴즈를 생성하세요.

출력은 반드시 JSON 객체 ONLY.
ox 퀴즈는 3개 출제, 객관식 퀴즈는 1개 출제.
객관식 퀴즈는 4지선다, 정답은 1개.

구조의 경우 해당 형식 대로 출력은 하지만 내용은 매번 바뀔 수 있음.
ox 퀴즈의 경우 정답이 O일 수도 X일 수도 있음.
다지선다 퀴즈의 정답 위치도 매번 바뀔 수 있음.
구조:
{{
  "quizzes": [
    {{
      "type": "keyword",
      "question": "요약문 속 핵심 키워드 빈칸 문제",
      "options": [
        {{ "text": "정답키워드1", "is_answer": true, "feedback": "정답 이유" }},
        {{ "text": "정답키워드2", "is_answer": true, "feedback": "정답 이유" }},
        {{ "text": "오답키워드1", "is_answer": false, "feedback": "오답 이유" }},
        {{ "text": "오답키워드2", "is_answer": false, "feedback": "오답 이유" }}
      ]
    }},
    {{
      "type": "ox",
      "question": "기사 내용 참/거짓 문제",
      "options": [
        {{ "text": "O", "is_answer": true, "feedback": "정답 이유" }},
        {{ "text": "X", "is_answer": false, "feedback": "오답 이유" }}
      ]
    }},
    {{
      "type": "multiple_choice",
      "question": "기사 요약 기반 객관식 문제",
      "options": [
        {{ "text": "보기1", "is_answer": true, "feedback": "정답 이유" }},
        {{ "text": "보기2", "is_answer": false, "feedback": "오답 이유" }},
        {{ "text": "보기3", "is_answer": false, "feedback": "오답 이유" }},
        {{ "text": "보기4", "is_answer": false, "feedback": "오답 이유" }}
      ]
    }}
  ]
}}

기사 요약문:
{latest_article.get("summary")}
"""

resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}]
)

try:
    quiz_json = json.loads(resp.choices[0].message.content)
except Exception as e:
    print("JSON 파싱 실패:", e)
    quiz_json = {"quizzes": []}

# === articles 붙여주기 ===
final_output = {
    "articles": [latest_article],
    "quizzes": quiz_json["quizzes"]
}

print(json.dumps(final_output, ensure_ascii=False, indent=2))


JSON 파싱 실패: Expecting value: line 1 column 1 (char 0)
{
  "articles": [
    {
      "id": "1615114173493154068",
      "sections": [
        "politics"
      ],
      "title": "장동혁 \"특검 압수수색, 최소한의 증거만 임의제출 되도록 협의\"",
      "publisher": "뉴스1",
      "author": "홍유진",
      "summary": "\"협의 사항에 대해 따로 보고 받은 바 없어\" \"공정노사법, 당론으로 추진할지는 원내대표와 협의할 것\" (서울=뉴스1) 박소은 홍유진 기자 = 장동혁 국민의힘 대표는 4일 12·3 비상계엄 관련 내란·외환 사건을 수사하는 조은석 특별검사팀이 자당의 원내대표실과 원내행정국 압수수색을 시도하는 것에 대해 \"최소한의 범위에서, 최소한의 증거들만 임의제출 될 수 있도록 협의해 나가겠다\"고 강조했다.\n\n법원에서도 특별재판부, 인민재판부에 대해서는 심각한 우려를 표하고 있다\"며 \"대한민국이 민주화 이후에, 그리고 지금의 헌법을 가진 이후에 헌법 규정에도 없는 특별재판부를 만든다고 하는 상상 자체가 믿기 어렵다\"고 했다.\n\n그러면서 \"결국 특검이 성과 내지 못하고, 그간의 내란 정당 몰이가 성과 없이 끝나게 될 것으로  ...",
      "highlight": null,
      "score": null,
      "image_url": "https://ddi-cdn.deepsearch.com/news/politics/2025/09/04/1615114173493154068/000-9ecc9ec464b060c96c9ad3a10cebd95016014bbf.jpg",
      "thumbnail_url": "https://ddi-cdn.deepsearch.com/news_thumbnail/politics/2025/09/04/161511417349

# N 단계 실행

In [19]:
import json
from langgraph.graph import StateGraph, END
from typing import TypedDict, Dict

# === 상태 정의 ===
class QuizState(TypedDict):
    stage: str
    result: Dict

# === Novice Subnodes ===
def background_knowledge(state: QuizState) -> QuizState:
    print("🧠 [Background Node 실행 중]")
    state["result"]["background"] = "배경지식 생성 완료"
    return state

def novice_quiz(state: QuizState) -> QuizState:
    print("📝 [NoviceQuiz Node 실행 중]")
    state["result"]["novice_quiz"] = "초급 퀴즈 생성 완료"
    return state

# === 그래프 생성 ===
graph = StateGraph(QuizState)

graph.add_node("Background", background_knowledge)
graph.add_node("NoviceQuiz", novice_quiz)

# === 노드 간 연결 ===
graph.add_edge("Background", "NoviceQuiz")  # Background → NoviceQuiz
graph.add_edge("NoviceQuiz", END)            # NoviceQuiz → END

# === Entry Point ===
graph.set_entry_point("Background")

# === 그래프 실행 ===
app = graph.compile()

print("\n🚀 그래프 실행 시작\n")

initial_state: QuizState = {"stage": "novice", "result": {}}
final_state = app.invoke(initial_state)

# === JSON 전체 출력 ===
print("\n✅ 그래프 실행 완료 (전체 상태 JSON)")
print(json.dumps(final_state, ensure_ascii=False, indent=2))



🚀 그래프 실행 시작

🧠 [Background Node 실행 중]
📝 [NoviceQuiz Node 실행 중]

✅ 그래프 실행 완료 (전체 상태 JSON)
{
  "stage": "novice",
  "result": {
    "background": "배경지식 생성 완료",
    "novice_quiz": "초급 퀴즈 생성 완료"
  }
}
